In [1]:
import ITensors:op
import Pkg; Pkg.add("HDF5")
import Pkg; Pkg.add("Plots")
import Pkg; Pkg.add("PyFormattedStrings")
import Pkg; Pkg.add("LinearAlgebra")
import Pkg; Pkg.add("LaTeXStrings")

   Resolving package versions...
  No Changes to `~/.julia/environments/v1.10/Project.toml`
  No Changes to `~/.julia/environments/v1.10/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.10/Project.toml`
  No Changes to `~/.julia/environments/v1.10/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.10/Project.toml`
  No Changes to `~/.julia/environments/v1.10/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.10/Project.toml`
  No Changes to `~/.julia/environments/v1.10/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.10/Project.toml`
  No Changes to `~/.julia/environments/v1.10/Manifest.toml`


In [2]:
using ITensors, ITensorMPS
import ITensors:op
using HDF5
using Plots
using PyFormattedStrings
using LinearAlgebra
using ITensorTDVP: tdvp

ITensors.disable_warn_order()

LoadError: ArgumentError: Package ITensorTDVP not found in current path.
- Run `import Pkg; Pkg.add("ITensorTDVP")` to install the ITensorTDVP package.

In [3]:
#include("overload_Spin=3h.jl")
# overloadings for SiteType 3/2
#----------------------------------------------
ITensors.space(::SiteType"S=3/2") = 4

# operator Sz
#-------------------
ITensors.op(::OpName"Sz",::SiteType"S=3/2") =
[+3/2   0    0    0
  0    +1/2  0    0
  0     0   -1/2  0
  0     0    0   -3/2]

# operator S+
#-------------------
ITensors.op(::OpName"S+",::SiteType"S=3/2") =
[0  √3  0   0 
 0  0   √4  0
 0  0   0   √3
 0  0   0   0]

# operator S-
#-------------------
ITensors.op(::OpName"S-",::SiteType"S=3/2") =
[0  0   0   0
 √3 0   0   0
 0  √4  0   0
 0  0   √3  0]

# z projection states
#-----------------------------------------
ITensors.state(::StateName"+3/2",::SiteType"S=3/2") = [1, 0, 0, 0]
ITensors.state(::StateName"+1/2",::SiteType"S=3/2") = [0, 1, 0, 0]
ITensors.state(::StateName"-1/2",::SiteType"S=3/2") = [0, 0, 1, 0]
ITensors.state(::StateName"-3/2",::SiteType"S=3/2") = [0, 0, 0, 1]

In [4]:
# construction of the MPO hamiltonian
#------------------------------------------
function Hamiltonian(sites)
    ops = OpSum()

    N = length(sites)
    # spin-spin interaction
    for k=1:N-1
        ops += J,   "Sz",k, "Sz",k+1
        ops += J/2, "S+",k, "S-",k+1
        ops += J/2, "S-",k, "S+",k+1
    end
    #make it a ring
    ops += J,   "Sz",N, "Sz",1
    ops += J/2, "S+",N, "S-",1
    ops += J/2, "S-",N, "S+",1

    # spin-B interaction
    for k=1:N
        ops += -B, "Sz",k
    end
    
    H = MPO(ops,sites)
    #--------------------------
    return H
end

Hamiltonian (generic function with 1 method)

In [5]:
function Hamiltonian1(sites)
    ops = OpSum()

    N = length(sites)
    # spin-spin interaction
    for k=1:N-1
        ops += J,   "Sz",k, "Sz",k+1
        ops += J/2, "S+",k, "S-",k+1
        ops += J/2, "S-",k, "S+",k+1
    end
    #make it a ring
    ops += J,   "Sz",N, "Sz",1
    ops += J/2, "S+",N, "S-",1
    ops += J/2, "S-",N, "S+",1

    # spin-B interaction
    for k=1:2
        ops += -B, "Sz",k
    end
    ops += -B1, "Sz",3
    
    for k=4:N
        ops += -B, "Sz",k
    end
    
    H = MPO(ops,sites)
    #--------------------------
    return H
end

Hamiltonian1 (generic function with 1 method)

In [6]:

#four point RDM of sites AB, CD
# AB adjacent, CD adjacent
#------------------------------------------
function get_4pdm(wf::MPS,locs)
    psi = copy(wf)
    A,C = locs

    orthogonalize!(psi,A)
    ket = psi[A]
    for k in A+1:C+1
        ket *= psi[k]
    end
    rho = prime(ket,"Site") * dag(ket)

    if C-A>2
        inds_list = inds(rho)
        trinds1 = collect(3 : C-A)
        trinds2 = trinds1[end] .+ trinds1 .+ 2
        for idx in 1:length(trinds1)
            rho = rho*delta(inds_list[trinds1[idx]],inds_list[trinds2[idx]])
        end
    end
    #-------------------
    return rho
end

get_4pdm (generic function with 1 method)

In [7]:
#   calc_Eigenvalues of density matrix
#-----------------------------------
function calc_Eigenvalues(density_matrix)
    egn_val,_ = eigen(density_matrix,ishermitian=true)
    #-----------------------------------
    return diag(array(egn_val))
end


#   eigenvalue based generic properties of a density matrix
#-----------------------------------
function calc_Norm(density_matrix)
    rho = copy(density_matrix)

    egn_val = calc_Eigenvalues(rho)
    Norm = sum(egn_val)
    Purity = sum(egn_val.^2)

    #-----------------------------------
    return [Norm,Purity]
end

#   eigenvalue based generic properties of a density matrix
#-----------------------------------
function calc_SvN(density_matrix)
    rho = copy(density_matrix)

    egn_val = calc_Eigenvalues(rho)
    SvN = sum( [ - lam*log(lam) for lam in egn_val if lam > 0 ] )

    #-----------------------------------
    return SvN
end


calc_SvN (generic function with 1 method)

In [8]:
#define the parameters of the hamiltonian and constuct MPO
#----------------------------------------------------------------------------
J = 1 
B = 0.1
B1 = 0.001
nsweeps = 10
maxdim = [100, 200, 500, 1000, 2000, 5000, 10000, 25580]
cutoff = [1E-11]

#--------------------
N = 14
#--------------------

site_list = collect(1:N) 
sites = siteinds( k->isodd(k) ? "S=1/2" : "S=3/2", N ) 
#sites = siteinds("S=1/2", N)

H = Hamiltonian(sites) 

# ground state and first excited state
#--------------------------------------------
energy0,psi0 = dmrg(H,random_mps(sites);nsweeps,maxdim,cutoff,ishermitian=true) ;
#energy1,psi1 = dmrg(H,[psi0],random_mps(sites);nsweeps,maxdim,cutoff,ishermitian=true) ;

After sweep 1 energy=-13.10697622635777  maxlinkdim=8 maxerr=3.00E-16 time=19.525
After sweep 2 energy=-14.004634638375117  maxlinkdim=32 maxerr=2.78E-16 time=0.123
After sweep 3 energy=-14.271121918672296  maxlinkdim=127 maxerr=6.02E-12 time=0.492
After sweep 4 energy=-14.406945462513887  maxlinkdim=387 maxerr=9.93E-12 time=5.386
After sweep 5 energy=-14.453490660154767  maxlinkdim=582 maxerr=9.99E-12 time=16.514
After sweep 6 energy=-14.4662208143466  maxlinkdim=521 maxerr=9.97E-12 time=16.250
After sweep 7 energy=-14.469489362709558  maxlinkdim=416 maxerr=9.94E-12 time=10.900
After sweep 8 energy=-14.470358761533172  maxlinkdim=309 maxerr=9.94E-12 time=6.755
After sweep 9 energy=-14.470582980793687  maxlinkdim=241 maxerr=9.99E-12 time=4.130
After sweep 10 energy=-14.47063984504539  maxlinkdim=189 maxerr=9.93E-12 time=2.181


In [9]:
H1 = Hamiltonian1(sites) 
ϕ = tdvp(
    H1,
    2.0im,
    psi0;
    time_step=1.0im,
    maxdim=30,
    cutoff=1e-10,
    normalize=true,
    reverse_step=false,
    outputlevel=1,
  )

After sweep 1: maxlinkdim=30 maxerr=8.06E-08 current_time=0.0 + 1.0im time=47.425
After sweep 2: maxlinkdim=30 maxerr=2.63E-07 current_time=0.0 + 2.0im time=4.078


MPS
[1] ((dim=2|id=284|"Link,l=1"), (dim=2|id=55|"S=1/2,Site,n=1"))
[2] ((dim=8|id=839|"Link,l=2"), (dim=4|id=142|"S=3/2,Site,n=2"), (dim=2|id=284|"Link,l=1"))
[3] ((dim=16|id=142|"Link,l=3"), (dim=2|id=39|"S=1/2,Site,n=3"), (dim=8|id=839|"Link,l=2"))
[4] ((dim=30|id=872|"Link,l=4"), (dim=4|id=347|"S=3/2,Site,n=4"), (dim=16|id=142|"Link,l=3"))
[5] ((dim=30|id=579|"Link,l=5"), (dim=2|id=507|"S=1/2,Site,n=5"), (dim=30|id=872|"Link,l=4"))
[6] ((dim=30|id=981|"Link,l=6"), (dim=4|id=665|"S=3/2,Site,n=6"), (dim=30|id=579|"Link,l=5"))
[7] ((dim=30|id=971|"Link,l=7"), (dim=2|id=496|"S=1/2,Site,n=7"), (dim=30|id=981|"Link,l=6"))
[8] ((dim=30|id=644|"Link,l=8"), (dim=4|id=533|"S=3/2,Site,n=8"), (dim=30|id=971|"Link,l=7"))
[9] ((dim=30|id=691|"Link,l=9"), (dim=2|id=172|"S=1/2,Site,n=9"), (dim=30|id=644|"Link,l=8"))
[10] ((dim=30|id=20|"Link,l=10"), (dim=4|id=887|"S=3/2,Site,n=10"), (dim=30|id=691|"Link,l=9"))
[11] ((dim=29|id=696|"Link,l=11"), (dim=2|id=165|"S=1/2,Site,n=11"), (dim=30|id=20|"Link

In [10]:
exp_Sz = expect( ϕ, "Sz" )

14-element Vector{Float64}:
 -0.3586900942901282
  1.3615370394926782
 -0.3643328191889743
  1.3624580744523165
 -0.3586763260100364
  1.3574784353418903
 -0.3577294218628015
  1.3576805785457724
 -0.3574524110286059
  1.3568881128019454
 -0.3570085300348646
  1.356926399771175
 -0.35736424859764615
  1.3581337564292253

In [11]:
sum=0
for i in 1:length(exp_Sz)
    sum+=exp_Sz[i]
end

sum

6.999848545821946

In [12]:
using Plots
using LaTeXStrings
plot( expect(ϕ,"Sz"),marker=:circle )

GKS: dlopen(/Users/Hank/.julia/artifacts/66ebf8a541da7166677a6e2ac6b8277592d32044/lib/cairoplugin.so, 1): Library not loaded: @rpath/libcairo.2.dylib
  Referenced from: /Users/Hank/.julia/artifacts/66ebf8a541da7166677a6e2ac6b8277592d32044/lib/cairoplugin.so
  Reason: Incompatible library version: cairoplugin.so requires version 11603.0.0 or later, but libcairo.2.dylib provides version 2.0.0


In [13]:
# TDVP parameters
time_step = 1.0im
maxdim = 30
cutoff = 1e-10
normalize = true
reverse_step = false
outputlevel = 1


# Initialize lists to store total times and sums of expectation values
total_times = [1.0im + (i-1)*time_step for i in 1:15]
sum_values = []

for total_time in total_times
    # Evolve the state
    psi = tdvp(H1, total_time, psi0; time_step=time_step, maxdim=maxdim, cutoff=cutoff, normalize=normalize, reverse_step=reverse_step, outputlevel=outputlevel)
    
    # Compute the expectation value of Sz
    exp_Sz = expect(psi, "Sz")
    
    # Sum the expectation values
    sum=0
    
    for i in 1:length(exp_Sz)
        sum+=exp_Sz[i]
    end
    
    # Store the sum
    push!(sum_values, sum)
end


# Plot the sum of expectation values vs total time
plot(abs.(total_times), sum_values, xlabel="time", ylabel="< Sz>", lw=2, label="Sum(Sz)")



After sweep 1: maxlinkdim=30 maxerr=8.06E-08 current_time=0.0 + 1.0im time=9.044
After sweep 1: maxlinkdim=30 maxerr=8.06E-08 current_time=0.0 + 1.0im time=8.74
After sweep 2: maxlinkdim=30 maxerr=2.63E-07 current_time=0.0 + 2.0im time=4.519
After sweep 1: maxlinkdim=30 maxerr=8.06E-08 current_time=0.0 + 1.0im time=8.489
After sweep 2: maxlinkdim=30 maxerr=2.63E-07 current_time=0.0 + 2.0im time=4.066
After sweep 3: maxlinkdim=30 maxerr=4.06E-07 current_time=0.0 + 3.0im time=4.154
After sweep 1: maxlinkdim=30 maxerr=8.06E-08 current_time=0.0 + 1.0im time=8.765
After sweep 2: maxlinkdim=30 maxerr=2.63E-07 current_time=0.0 + 2.0im time=4.167
After sweep 3: maxlinkdim=30 maxerr=4.06E-07 current_time=0.0 + 3.0im time=4.351
After sweep 4: maxlinkdim=30 maxerr=4.75E-07 current_time=0.0 + 4.0im time=4.513
After sweep 1: maxlinkdim=30 maxerr=8.06E-08 current_time=0.0 + 1.0im time=8.39
After sweep 2: maxlinkdim=30 maxerr=2.63E-07 current_time=0.0 + 2.0im time=4.097
After sweep 3: maxlinkdim=30 m

After sweep 11: maxlinkdim=30 maxerr=1.60E-07 current_time=0.0 + 11.0im time=4.065
After sweep 12: maxlinkdim=30 maxerr=1.45E-07 current_time=0.0 + 12.0im time=4.096
After sweep 13: maxlinkdim=30 maxerr=1.23E-07 current_time=0.0 + 13.0im time=4.115
After sweep 14: maxlinkdim=30 maxerr=1.03E-07 current_time=0.0 + 14.0im time=4.495
After sweep 1: maxlinkdim=30 maxerr=8.06E-08 current_time=0.0 + 1.0im time=8.478
After sweep 2: maxlinkdim=30 maxerr=2.63E-07 current_time=0.0 + 2.0im time=4.098
After sweep 3: maxlinkdim=30 maxerr=4.06E-07 current_time=0.0 + 3.0im time=4.467
After sweep 4: maxlinkdim=30 maxerr=4.75E-07 current_time=0.0 + 4.0im time=4.029
After sweep 5: maxlinkdim=30 maxerr=4.22E-07 current_time=0.0 + 5.0im time=4.091
After sweep 6: maxlinkdim=30 maxerr=3.61E-07 current_time=0.0 + 6.0im time=4.098
After sweep 7: maxlinkdim=30 maxerr=3.42E-07 current_time=0.0 + 7.0im time=5.388
After sweep 8: maxlinkdim=30 maxerr=2.92E-07 current_time=0.0 + 8.0im time=5.146
After sweep 9: maxli

In [14]:
sum_values

15-element Vector{Any}:
 6.999846178829365
 6.999848545821946
 6.999878023972382
 6.9998933020496565
 6.999913406077656
 6.9999295777056325
 6.999938663278176
 6.999949858995768
 6.999957870999202
 6.999965700599779
 6.999969542306212
 6.999974866271681
 6.999979218240677
 6.999984755787491
 6.999986758784351